In [ ]:
from sympy import floor, oo
from sympy.stats import P, Normal
from scipy import stats
import plotly.graph_objects as go
from numpy import transpose, sqrt

classes = [[-oo,57], [57,62], [62,67], [67,72], [72,77], [77,82], [82,oo]]
delta_t_amounts = [5,13,20,26,24,6,6]
n = sum(delta_t_amounts)
t_mean = 68.56
sigma = 7.73072
alpha = 0.05

class sample:
    n = n
    lamd = 1/t_mean

    def __init__(self, lowerLimit_closed, upperLimit_open, nᵢ):
        self.lowerLimit = lowerLimit_closed
        self.upperLimit = upperLimit_open
        self.nᵢ = nᵢ
        self.X = Normal('X', t_mean, sigma)
        self.pᵢ = float(P(self.X <= self.upperLimit) - P(self.X <= self.lowerLimit))
        self.nᵢˣ = sample.n * self.pᵢ
        self.equation = (self.nᵢ - self.nᵢˣ)**2/self.nᵢˣ

def sum_col(name, table_data):
    return sum([table_data[i][name] for i, item in enumerate(table_data)])

if n != sum(delta_t_amounts):
    raise IOError("delta_t_amounts and n are not equal, possibly wrong input")

s = [sample(delta_t_class[0], delta_t_class[1], delta_t_amounts[i]) for i, delta_t_class in enumerate(classes)]

table_data = []
for i, _sample in enumerate(s):
    samples = {}
    samples['class'] = f"[{_sample.lowerLimit},{_sample.upperLimit})"
    samples['nᵢ'] = _sample.nᵢ
    samples['pᵢ'] = _sample.pᵢ
    samples['nᵢˣ'] = _sample.nᵢˣ
    samples['equation'] = _sample.equation
    table_data.append(samples)
table_data.append({'class': 'sum', 'nᵢ': sum_col('nᵢ', table_data), 'pᵢ': sum_col('pᵢ', table_data), 'nᵢˣ': sum_col('nᵢˣ', table_data), 'equation': sum_col('equation', table_data)})

f = len(delta_t_amounts) -1
quantile = 1-alpha
degrees = f-2 # 2 assumptions -> degrees -2
c = stats.chi2.ppf(quantile, degrees)
z = table_data[-1]['equation']

print(f'estimated value 𝜆̂ = {1/t_mean}')
print(f"class {table_data[0]['class']}: pᵢ = {table_data[0]['pᵢ']}")
print(f"class {table_data[1]['class']}: equation = {table_data[1]['equation']}")
print(f"class {table_data[-2]['class']}: nᵢˣ = {table_data[-2]['nᵢˣ']}")
print(f"test value: z = {z}")
print(f'c = {c} is the {quantile:1f}-quantile of the 𝜒²-distribution with f = {degrees} degrees')
print(f"the sample {'seems' if z < c else 'does not seem'} to be equally distributed with a {alpha*100}% error rate")

fig = go.Figure(data=[go.Table(header=dict(values=list(samples.keys())),
                 cells=dict(values=transpose([list(item.values()) for item in table_data]), align="left"))
                    ])
fig.show()